# UAI 2018 SPC Invitations (and setup)
2017-11-21

This notebook provides some high-level guidance on using OpenReview for UAI 2018.

In [ ]:
'''
use the openreview python library to interact with OpenReview. You can get it with `pip install openreview-py`.
'''

import openreview
client = openreview.Client()
print client.baseurl

## OpenReview basics

OpenReview is based on three main object types: Groups, Notes, and Invitations

Groups represent people or entities in OpenReview. A group's `members` field can contain user IDs, emails, or other groups.

Notes represent data records in the system; for example, a record of a submitted paper, or a record of a comment on that paper, or a record that confirms whether or not a user accepts the responsibility to serve as a reviewer.

Invitations can be thought of as requests for Notes. Invitations specify a general template for a Note. When a Note is posted, it must be posted in response to an Invitation, and it must adhere to the requirements specified by that Invitation.

In the block below, we create the basic infrastructure to allow SPC members to accept or decline SPC membership.

## Posting the conference's Root group

Before we can do anything else, we need to create a group that represents the conference itself. It will have the ID "auai.org/UAI/2018". 

In [ ]:
# post the root group
client.post_group(openreview.Group(**{
        'id': 'auai.org/UAI/2018',
        'readers': ['everyone'],
        'writers': ['~Super_User1'],
        'signatures': ['~Super_User1'],
        'signatories': ['auai.org/UAI/2018'],
        'members': []
    }))

explanation of parameters:
- readers: a list of groups that have permission to view the group.
- writers: a list of groups that have permission to modify the group.
- signatures: a list of groups that have previously modified the group (in practice: the last group to have modified this group)
- signatories: a list of groups that has permission to sign other groups, notes, and invitations using this group's ID. (you can think of this as the list of people or groups that can "speak for" this group)
- members: a list of groups that are members of this group (membership does not imply readership, writership, or signatory permission!)

You can examine any group that you have permission to view by going to the following URL: https://openreview.net/groups?id=GROUPID (the group is represented in JSON; you may find it convenient to install a browser extension that parses JSON so that you can read it more easily)

## Posting the other groups needed for recruiting the SPCs

In the following cell, we'll create:
- a group for the Program Chairs.
- multiple groups for the Senior Program Committee, which we'll use to track who has accepted or declined

In [ ]:
# post the program chairs group
client.post_group(openreview.Group(**{
        'id': 'auai.org/UAI/2018/Program_Chairs',
        'readers': ['auai.org/UAI/2018','auai.org/UAI/2018/Program_Chairs'],
        'writers': ['auai.org/UAI/2018'],
        'signatures': ['auai.org/UAI/2018'],
        'signatories': ['auai.org/UAI/2018/Program_Chairs'],
        'members': ['~Yin_Cheng_Ng1','~Ricardo_Silva1','~Amir_Globerson1']
    }))

# post the SPC group(s)
client.post_group(openreview.Group(**{
        'id': 'auai.org/UAI/2018/Senior_Program_Committee',
        'readers': ['auai.org/UAI/2018','auai.org/UAI/2018/Program_Chairs','auai.org/UAI/2018/Senior_Program_Committee'],
        'writers': ['auai.org/UAI/2018'],
        'signatures': ['auai.org/UAI/2018'],
        'signatories': [],
        'members': []
    }))

client.post_group(openreview.Group(**{
        'id': 'auai.org/UAI/2018/Senior_Program_Committee/Invited',
        'readers': ['auai.org/UAI/2018','auai.org/UAI/2018/Program_Chairs'],
        'writers': ['auai.org/UAI/2018'],
        'signatures': ['auai.org/UAI/2018'],
        'signatories': [],
        'members': []
    }))

client.post_group(openreview.Group(**{
        'id':'auai.org/UAI/2018/Senior_Program_Committee/Declined',
        'readers': ['auai.org/UAI/2018','auai.org/UAI/2018/Program_Chairs'],
        'writers': ['auai.org/UAI/2018'],
        'signatures': ['auai.org/UAI/2018'],
        'signatories': [],
        'members': []
    }))

## Creating the recruitment Invitation

SPC recruitment works like this:

We create an Invitation that accepts three fields: an email address, a response (yes or no), and a unique hash key based on the email address.

The Invitation has two special fields: `process` and `web`.

`process` defines a small piece of javascript that is executed when the Invitation is fulfilled. In this case, the file `recruitSPCProcess.js` checks to see that the hash key matches the email address, and checks the response (yes or no) of the response. If the response is "yes", the email address is added as a member to the group auai.org/UAI/2018/Senior_Program_Committee. Otherwise, they are added to the group auai.org/UAI/2018/Senior_Program_Committee/Declined.

`web` defines a small html page that the user sees when they accept or decline.

(This section is intentionally vague. We'll get into more details later.)

In [ ]:
# Create Senior_Program_Committee recruitment invitation/form
spc_invitation = openreview.Invitation(**{
        'id': 'auai.org/UAI/2018/-/SPC_Invitation',
        'readers': ['everyone'],
        'writers': ['auai.org/UAI/2018'],
        'signatures': ['auai.org/UAI/2018'],
        'process': '../process/recruitSPCProcess.js',
        'web': '../webfield/recruitSPCWebfield.html'
    })
                                       
spc_invitation.reply = {
    'content': {
        'email': {
            'description': 'email address',
            'order': 1,
            'value-regex': '.*@.*'
        },
        'key': {
            'description': 'Email key hash',
            'order': 2,
            'value-regex': '.{0,100}'
        },
        'response': {
            'description': 'Invitation response',
            'order': 3,
            'value-radio': ['Yes', 'No']
        }
    },
    'readers': {
        'values': ['OpenReview.net']
    },
    'signatures': {
        'values-regex': '\\(anonymous\\)'
    },
    'writers': {
        'values-regex': '\\(anonymous\\)'
    }
}
                                       
client.post_invitation(spc_invitation)

## Define the message

Sending emails through OpenReview is easy with the `openreview.Client`'s send_mail function. 

That being said, this message is special, because the accept/decline links that are included in it are specific to each individual email recipient. Because of that, we need to add some extra functionality.

See the definition of the new `send_mail` function in the cell below.

In [ ]:
subject = 'UAI 2018: Invitation to serve on the Senior Program Committee'

message = '''Dear {0},

As program chairs of UAI 2018, it is our pleasure to invite you to serve as a member of the Senior Program Committee (SPC) for the 2018 Uncertainty in AI Conference (UAI). The conference will be held in the San Francisco area in early August 2018.

UAI is the premier international conference on research related to representation, inference, learning and decision making in the presence of uncertainty, as they relate to the fields of artificial intelligence and machine learning. As a Senior Program Committee Member for the UAI 2018 conference you will help us select a high quality program for the conference. We count on your help and your expertise. 

To ACCEPT the invitation, please click on the following link:

{1}

To DECLINE the invitation, please click on the following link:

{2}

If you already have an OpenReview account, please make sure that the email address that received this message is linked to your account. 
If you do not already have an OpenReview account, please sign up at https://openreview.net/signup, making sure to add the email address that received this message to your profile.
Please email info@openreview.net with any questions about the signup process.

The timeline of SPC actions is the following:

1. By December 1st, 2017: Please respond by clicking above indicating whether you accept the SPC invitation or not.
2. March 4-9, 2018: Enter bids for submitted papers.
3. March 12-18: Suggest 3 to 5 reviewers for each paper assigned to you. 
4. March 19th to April 20th: Review period. If a reviewer drops out, we might need your help in finding additional reviewers for the paper.
5. April 13th to April 20th: Check reviews and chase reviewers for missing ones.
6. April 21st to May 3rd: Discussion period with reviewers, suggest further reviewers if necessary.
7. By May 6th, write a meta-review for each paper. 
10. May 7th - May 13th: Your meta-reviews will be used for making the final decisions. If we need additional input on a particular paper we might contact you.

We really hope you will be able to accept our invitation and help us select a high quality program for UAI 2018.  Please reply to this invitation by following the link at the beginning of this message, no later than December 1st, 2017.

Thanks in advance for your help!

Amir Globerson and Ricardo Silva
UAI 2018 Program Chairs
uai2018chairs@gmail.com'''

def send_mail(email, first, last):
    hashkey = client.get_hash(email.encode('utf-8'), "2810398440804348173") # the second argument here is just a big random number that the process function also knows about.
    url = client.baseurl+"/invitation?id=auai.org/UAI/2018/-/SPC_Invitation&email=" + email + "&key=" + hashkey + "&response="
    response = client.send_mail(subject, [email], message.format(first, url+"Yes", url+"No"))
    print "Mail response: ", response

## Send messages, add users to group
Finally, we can loop through the csv file of SPC members to invite, add them to the group of Invited SPCs, and send the messages:

In [ ]:
import csv
spc_invited = client.get_group('auai.org/UAI/2018/Senior_Program_Committee/Invited')
with open('../data/UAI2018_SPCs.csv') as f:
    for first, last, email in csv.reader(f):
        email = email.strip().lower()
        print first, email
        send_mail(email, first, last)
        spc_invited = client.add_members_to_group(spc_invited, email)
